In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
import pandas as pd

def load_housing_data(housing_path='data'):
    train_csv_path = os.path.join(housing_path, "train.csv")
    test_csv_path = os.path.join(housing_path, "test.csv")
    return pd.read_csv(train_csv_path), pd.read_csv(test_csv_path)

train, test = load_housing_data()

In [3]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [4]:
num_att = list(train.select_dtypes([np.number]).columns)

# att = ['OverallQual', 'GrLivArea']

to_remove = ['BsmtFinSF2', 'MoSold', '3SsnPorch', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold',
            'PoolArea', 'OverallCond', 'MSSubClass', 'ScreenPorch']
for t in to_remove:
    num_att.remove(t)
num_att.remove('Id')
num_att.remove('SalePrice')
print(num_att)

['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch']


In [5]:
train.info()
# train.select_dtypes(include=['object'])
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [6]:
cat_att = ['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 'Foundation', 
           'PavedDrive', 'SaleCondition']
# cat_att = ['Neighborhood', 'BldgType']
print(cat_att)

['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 'Foundation', 'PavedDrive', 'SaleCondition']


In [7]:
for a in num_att:
    median = train[a].median()
    train[a].fillna(median, inplace=True)
    
#     median = test[a].median()
    test[a].fillna(median, inplace=True)

In [8]:
train[num_att].info()
test[num_att].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 25 columns):
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
GarageYrBlt      1460 non-null float64
GarageCars       1460 non-null int64
GarageArea       1460 non-null int64
WoodDeckSF       1460 non-null int64
OpenPorchSF      1460 non-null int64

In [9]:
train[cat_att].info()
test[cat_att].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 8 columns):
Neighborhood     1460 non-null object
BldgType         1460 non-null object
Heating          1460 non-null object
HeatingQC        1460 non-null object
CentralAir       1460 non-null object
Foundation       1460 non-null object
PavedDrive       1460 non-null object
SaleCondition    1460 non-null object
dtypes: object(8)
memory usage: 91.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 8 columns):
Neighborhood     1459 non-null object
BldgType         1459 non-null object
Heating          1459 non-null object
HeatingQC        1459 non-null object
CentralAir       1459 non-null object
Foundation       1459 non-null object
PavedDrive       1459 non-null object
SaleCondition    1459 non-null object
dtypes: object(8)
memory usage: 91.3+ KB


In [10]:
train_y = train["SalePrice"].copy()
# train = train.drop("SalePrice", axis=1)
# train = train.drop("Id", axis=1)
# test = test.drop("Id", axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [11]:
train2 = train[num_att + cat_att]
train2.info()
test2 = test[num_att + cat_att]
test2.info()

merged_df = pd.concat([train2, test2])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 33 columns):
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
GarageYrBlt      1460 non-null float64
GarageCars       1460 non-null int64
GarageArea       1460 non-null int64
WoodDeckSF       1460 non-null int64
OpenPorchSF      1460 non-null int64

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
train_prepared = num_pipeline.fit_transform(train[num_att])
test_prepared  = num_pipeline.transform(test[num_att])


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_att),
        ("cat", OneHotEncoder(), cat_att),
    ])

train_test_full_prepared = full_pipeline.fit_transform(merged_df)
train_full_prepared = full_pipeline.transform(train2)
test_full_prepared = full_pipeline.transform(test2)


In [13]:
print(train_prepared.shape)
print(test_prepared.shape)
print(train_full_prepared.shape)
print(test_full_prepared.shape)

(1460, 25)
(1459, 25)
(1460, 83)
(1459, 83)


In [14]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=200, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

housing_predictions = forest_reg.predict(train_full_prepared)
forest_mse = mean_squared_error(train_y, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

10929.950392645498

In [16]:
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("scores:", scores)
    print("mean:", scores.mean())
    print("std:", scores.std())

forest_reg = RandomForestRegressor(n_estimators=200, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

forest_scores = cross_val_score(forest_reg, train_full_prepared, train_y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

print(pd.Series(forest_rmse_scores).describe())

scores: [26242.38594706 27768.57521952 22414.68488182 40024.65687082
 32597.7336345  25786.72889609 24244.92819253 24578.87432989
 37465.08376339 27211.27599828]
mean: 28833.492773389975
std: 5603.892087358005
count       10.000000
mean     28833.492773
std       5907.020919
min      22414.684882
25%      24880.837971
50%      26726.830973
75%      31390.444031
max      40024.656871
dtype: float64


In [17]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # sprawdza 12 (3×4) kombinacji hiperparametrów
    {'n_estimators': [10, 30, 100, 200], 'max_features': [4, 8, 16, 32, 64, 83]},
    # następnie sprawdza 6 (2×3) kombinacji z wyłączonym parametrem bootstrap (False)
    {'bootstrap': [False], 'n_estimators': [10, 30, 100, 200], 'max_features': [4, 8, 16, 32, 64, 83]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# przeprowadza proces uczenia na pięciu podzbiorach, czyli łącznie (12+6)*5=90 przebiegów 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(train_full_prepared, train_y)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'max_features': [4, 8, 16, 32, 

In [18]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 16, 'n_estimators': 100}

In [19]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=16, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [20]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

33995.27746995363 {'max_features': 4, 'n_estimators': 10}
31538.850351978188 {'max_features': 4, 'n_estimators': 30}
29897.779557645343 {'max_features': 4, 'n_estimators': 100}
30218.84636539386 {'max_features': 4, 'n_estimators': 200}
32129.735660313112 {'max_features': 8, 'n_estimators': 10}
30561.010664415913 {'max_features': 8, 'n_estimators': 30}
29495.329738438177 {'max_features': 8, 'n_estimators': 100}
29373.97382859127 {'max_features': 8, 'n_estimators': 200}
31532.99804160975 {'max_features': 16, 'n_estimators': 10}
30149.39142025913 {'max_features': 16, 'n_estimators': 30}
29008.779272096337 {'max_features': 16, 'n_estimators': 100}
29006.69936943944 {'max_features': 16, 'n_estimators': 200}
30254.686509298208 {'max_features': 32, 'n_estimators': 10}
29878.527281645554 {'max_features': 32, 'n_estimators': 30}
28975.638497014344 {'max_features': 32, 'n_estimators': 100}
28835.908475046774 {'max_features': 32, 'n_estimators': 200}
30892.8978415971 {'max_features': 64, 'n_estim

In [23]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([1.26315094e-02, 2.64691225e-02, 1.76080274e-01, 6.01125810e-02,
       2.92902122e-02, 9.11693491e-03, 3.02650603e-02, 9.45565244e-03,
       7.65735031e-02, 5.78460810e-02, 4.33834145e-02, 1.33262781e-01,
       3.45498034e-03, 1.86637584e-02, 3.90177239e-03, 5.94777831e-03,
       2.11853601e-03, 2.12988395e-02, 1.96464464e-02, 2.96471883e-02,
       1.04819792e-01, 5.55749005e-02, 6.52637282e-03, 9.08010272e-03,
       1.52808887e-03, 1.32693891e-04, 1.43565658e-06, 4.35769159e-05,
       2.68278999e-04, 5.84279333e-04, 5.80528896e-04, 2.11774598e-03,
       2.21313249e-03, 2.29345867e-04, 2.11693294e-04, 1.12314475e-04,
       1.82767699e-04, 6.22920310e-04, 2.05740363e-05, 3.24831066e-04,
       2.34162093e-03, 2.07116944e-03, 1.51024307e-03, 1.42303949e-04,
       4.19652092e-04, 3.44959677e-04, 6.66447908e-04, 1.38547048e-03,
       3.78021959e-04, 1.91489050e-04, 1.64672066e-03, 1.32252029e-04,
       8.99202496e-04, 1.57954459e-04, 4.14347329e-04, 4.98493403e-07,
      

In [24]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_att + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.1760802744743346, 'OverallQual'),
 (0.1332627814305419, 'GrLivArea'),
 (0.10481979192012945, 'GarageCars'),
 (0.07657350306270239, 'TotalBsmtSF'),
 (0.06011258103003999, 'YearBuilt'),
 (0.05784608095558156, '1stFlrSF'),
 (0.05557490054328089, 'GarageArea'),
 (0.043383414534335404, '2ndFlrSF'),
 (0.030265060287039455, 'BsmtFinSF1'),
 (0.029647188344437282, 'GarageYrBlt'),
 (0.029290212223838943, 'YearRemodAdd'),
 (0.026469122529973167, 'LotArea'),
 (0.02129883946813158, 'TotRmsAbvGrd'),
 (0.01964644637424258, 'Fireplaces'),
 (0.01866375836784845, 'FullBath'),
 (0.012631509379675937, 'LotFrontage'),
 (0.009455652444630024, 'BsmtUnfSF'),
 (0.009116934914751131, 'MasVnrArea'),
 (0.009080102717239573, 'OpenPorchSF'),
 (0.006526372824839534, 'WoodDeckSF'),
 (0.005947778310061869, 'BedroomAbvGr'),
 (0.0039017723896120644, 'HalfBath'),
 (0.0034549803366426125, 'BsmtFullBath'),
 (0.002341620933177509, 'NoRidge'),
 (0.0022131324916920487, 'Edwards'),
 (0.0021185360056457984, 'KitchenAbvGr'),

In [25]:
y_pred = grid_search.predict(test_full_prepared)
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': y_pred})

In [28]:
sub.to_csv("data/submission_script09_rf_gridsearch.csv", index=False)

In [27]:
sub.shape

(1459, 2)